# Notebook to generate and save toy data in a .npz format

In [1]:
# System imports
import math
from __future__ import print_function

# Standard imports
import sys
import numpy as np

# Python Externa Imports
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Local imports
from local_modules import generate_data, track_hit_coords, draw_from_params,drawMulti_from_params_pix  

# Magic
#%matplotlib inline
%matplotlib notebook

In [2]:
# Configuration
num_det_layers = 10
det_layer_size = 32
num_seed_layers = 0
avg_bkg_tracks = 0
noise_prob = 0.0
det_shape = (num_det_layers, det_layer_size, det_layer_size)

max_tracks = 6
num_tracks = 3

# Training parameters
batch_size = 256
epoch_size = 25600
num_epochs = 10

#batch_size = 64
#epoch_size = 64
#num_epochs = 3

In [3]:
def batch_generator(num_batch, det_shape=det_shape,
                    num_seed_layers=num_seed_layers,
                    avg_bkg_tracks=avg_bkg_tracks,
                    noise_prob=noise_prob):
    """Generator of toy data batches for training"""
    shape = (num_batch,) + det_shape
    while True:
        events, sig_tracks, params, a = generate_data(
            shape, num_seed_layers=num_seed_layers,
            avg_bkg_tracks=avg_bkg_tracks,
            noise_prob=noise_prob, verbose=False)
        train_target = sig_tracks.reshape(num_batch, det_shape[0], -1)
        # params includes the slopes and entry points.
        yield (events, train_target, params, a)

In [4]:
# Set random seed
np.random.seed(2010)

# Generate a fixed dataset of multi-track events
gen_single = batch_generator(batch_size)

train_events = np.zeros((epoch_size, 1, num_det_layers, det_layer_size, det_layer_size))
train_targets = -99 * np.ones((epoch_size, max_tracks, 4))
train_a = -99 * np.ones((epoch_size, max_tracks, 2*num_det_layers))
train_weights = np.zeros((epoch_size, max_tracks))
for n in range(epoch_size):
    tracks,train_hits,params,a = gen_single.next()
    train_events[n,0] = np.clip( sum( tracks[:num_tracks] ), 0, 1)
    event_targets = np.asarray(params[:num_tracks])
    event_a = np.asarray(a[:num_tracks])
    train_targets[n,:num_tracks] = event_targets[event_targets[:,0].argsort()] # sort by first column
    train_a[n,:num_tracks] = event_a[event_a[:,0].argsort()] # sort by first column
    train_weights[n,:num_tracks] = np.ones((num_tracks))


In [5]:
np.savez("/bigdata/atsaris/new/data1trk_256", events=train_events, target_slope_pix=train_a, 
        target_slope=train_targets, weights=train_weights, num_det_layers=num_det_layers,
        det_layer_size=det_layer_size, max_tracks=max_tracks, num_tracks=num_tracks,
        batch_size=batch_size, epoch_size=epoch_size, num_epochs=num_epochs)

In [6]:
np.random.seed(2017)

# Generate a fixed dataset of multi-track events
gen_single = batch_generator(batch_size)
epoch_size = 1000

train_events = np.zeros((epoch_size, 1, num_det_layers, det_layer_size, det_layer_size))
train_targets = -99 * np.ones((epoch_size, max_tracks, 4))
train_a = -99 * np.ones((epoch_size, max_tracks, 2*num_det_layers))
train_weights = np.zeros((epoch_size, max_tracks))
for n in range(epoch_size):
    tracks,train_hits,params,a = gen_single.next()
    train_events[n,0] = np.clip( sum( tracks[:num_tracks] ), 0, 1)
    event_targets = np.asarray(params[:num_tracks])
    event_a = np.asarray(a[:num_tracks])
    train_targets[n,:num_tracks] = event_targets[event_targets[:,0].argsort()] # sort by first column
    train_a[n,:num_tracks] = event_a[event_a[:,0].argsort()] # sort by first column
    train_weights[n,:num_tracks] = np.ones((num_tracks))

In [7]:
np.savez("/bigdata/atsaris/new/test1trk_256", events=train_events, target_slope_pix=train_a, 
        target_slope=train_targets, weights=train_weights, num_det_layers=num_det_layers,
        det_layer_size=det_layer_size, max_tracks=max_tracks, num_tracks=num_tracks,
        batch_size=batch_size, epoch_size=epoch_size, num_epochs=num_epochs)